In [ ]:
!pip install -U torch diffusers>=0.22.0 transformers accelerate ftfy triton safetensors
import diffusers
import torch

In [ ]:
# Download weights
!wget https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny.safetensors

In [ ]:
from diffusers import StableDiffusionXLPipeline

pipeline = diffusers.StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    torch_dtype=torch.float16, 
    variant="fp16"
).to("cuda")

In [ ]:
from controlnet_lite import ControlNetLLLite

# Load ControlNetLllite weights 
path = 'kohya_controllllite_xl_canny.safetensors'
controlnet = ControlNetLLLite(path)

In [ ]:
from PIL import Image
import numpy as np

# Load Control Image
image = diffusers.utils.load_image("https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/sd_controlnet/hf-logo.png")
# Convert the image to numpy array
control_image = np.array(image)

In [ ]:
conditioning_weight = 1
# Currently there is no way to control start and end of the conditioning
# As it can only be controled from the diffusers.DiffusionPipeline, and will require
# separate custom pipeline for Txt2Img, Img2Img etc.
# TODO: figure out smart way to bypass this

# Apply ControlNetLLLite to the pipeline
controlnet.apply(pipe=pipeline, cond=control_image, weight=conditioning_weight)

In [ ]:
with torch.inference_mode():
        image = pipeline(prompt="aerial view, a futuristic research complex in a bright foggy jungle, hard lighting", num_inference_steps=20).images[0]
image

In [ ]:
# To unload all controlnetLite modules from the pipeline
from controlnet_lite import clear_all_lllite
clear_all_lllite()

In [ ]:
with torch.inference_mode():
        image = pipeline(prompt="aerial view, a futuristic research complex in a bright foggy jungle, hard lighting", num_inference_steps=20).images[0]
image